In [2]:
 ## QuantConnect Imports
%load_ext autoreload
%autoreload 2

from clr import AddReference
AddReference("System")
AddReference('System.Memory')
AddReference("QuantConnect.Algorithm")
AddReference("QuantConnect.Api")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Configuration")
AddReference("QuantConnect.Configuration")
AddReference("QuantConnect.Research")
AddReference("QuantConnect.Indicators")

from System import *
from QuantConnect import *
from QuantConnect.Algorithm import * 
from QuantConnect.Api import *
from QuantConnect.Configuration import *
from QuantConnect.Data import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Research import *
from QuantConnect.Indicators import *


import datetime 
import matplotlib.pyplot as plt
import pandas as pd

# PT imports 
import numpy as np 
import matplotlib.pylab as plt 
from helper import *  # my rolling window , mk_momentum etc. 
import scipy as sp 

# ipython debugger. set breakpoint with `dbg.set_trace()` 
import IPython.core.debugger as dbg

# execute this to connect to QuantConnect API 
api = Api()
api.Initialize(
    Config.GetInt("job-user-id", 94240), 
    Config.Get("api-access-token", "f9683aa5c308894f2238228a5f3545badd2e4b11526685cee58bb1cecf7e9f30"),
    Config.Get("data-folder"))
print(api.Connected)


# Create an quantBook instance
qb = QuantBook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
True


In [79]:
class DataStreamException(Exception):
    pass

class CryptoStream:
    
    # base url and supported markets
    url = "https://www.quantconnect.com/data/tree/crypto/{}/{}/{}"
    markets = ("gdax" ,"bitfinex" )
       
    def __init__(self, data_folder, quantBook = None , debug = True):
        self.storage = data_folder
        self.qb = quantBook 
        self.debug = debug
    
    def format_tail(self, date, data_type ):
        dtype = "trade" if data_type == DataType.Trade else "quote"
        return "{}_{}.zip".format(date.strftime("%Y%m%d"), dtype)
    
    def download_one(self, path, ticker, market, resolution, date,  data_type = DataType.Trade) : 
        """
        download zips directly from https://www.quantconnect.com/data/tree/crypto/gdax/minute/etheur/20170530_trade.zip
        https://www.quantconnect.com/data/tree/crypto/<market>/<resolution>/<ticker>/<YYYYMMDD>_<type>.zip
        """
        if market.lower() not in self.markets: 
            raise DataStreamException("Unknown market type {}".format(market))
        # now attempt download 
        dtype = "trade" if data_type == DataType.Trade else "quote"
        base_url = self.url.format(market.lower(), resolution.lower(), ticker.lower())
        src  = base_url + "/" + self.format_tail(date, data_type)
        dest = path 
        try: 
            if self.debug:
                print("downloading ")
            fname = wget.download(src,dest)
            os.chmod(fname,  0o666)
        except Exception:
            raise False
        return True
    
    def add_one(self, ticker, market, resolution, date, data_type = DataType.Trade):
        # first check if data exist 
        if market.lower() not in self.markets: 
            raise DataStreamException("Unknown market type {}".format(market))
        # check if file already exists 
        path = os.path.join(os.path.abspath(self.storage),"crypto",market.lower(),resolution.lower(), ticker.lower())
        download = False
        if not os.path.exists(path):
            if self.debug : 
                print ("directory base path '{}' does not exist".format(path))
            os.makedirs(path,exist_ok=True)
            download = True
        elif not os.path.exists(path + "/" + self.format_tail(date, data_type)):
            download = True
        else: 
            print  ("data already exists.")
        # if we need to download something        
        if download: 
            status = self.download_one(path, ticker, market, resolution, date,  data_type)
            if status == False: 
                raise DataStreamException("could not download {} {}-resolution data for {}".format(ticker, resolution, date))
    
    def add_many(self, ticker, market, resolution, begin, end , data_type = DataType.Trade):
        diff = end-begin
        for i in range(diff.days):
            date = (begin+datetime.timedelta(i))
            if self.debug: 
                print("adding data for {}.{} on {}".format(ticker, resolution, date.strftime("%Y%m%d")))
                      
            self.add_one(ticker, market,resolution, date, data_type)
            

In [80]:
c_stream = CryptoStream(Config.Get("data-folder"))

In [81]:
c_stream.add_one("etheur", "gdax", "minute", datetime.datetime(2021,2,11))

data already exists.


In [82]:
start = datetime.datetime(2021,1,1)
end = datetime.datetime(2021,2,12)
c_stream.add_many("etheur","gdax","minute",start, end)

adding data for etheur.minute on 20210101
downloading 
adding data for etheur.minute on 20210102
downloading 
adding data for etheur.minute on 20210103
downloading 
adding data for etheur.minute on 20210104
downloading 
adding data for etheur.minute on 20210105
downloading 
adding data for etheur.minute on 20210106
downloading 
adding data for etheur.minute on 20210107
downloading 
adding data for etheur.minute on 20210108
downloading 
adding data for etheur.minute on 20210109
downloading 
adding data for etheur.minute on 20210110
downloading 
adding data for etheur.minute on 20210111
downloading 
adding data for etheur.minute on 20210112
downloading 
adding data for etheur.minute on 20210113
downloading 
adding data for etheur.minute on 20210114
downloading 
adding data for etheur.minute on 20210115
downloading 
adding data for etheur.minute on 20210116
downloading 
adding data for etheur.minute on 20210117
downloading 
adding data for etheur.minute on 20210118
downloading 
adding dat